<a href="https://colab.research.google.com/github/PokeMaster16/Machine-Learning-Projects/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [5]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-10-11 21:13:49--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  91.1MB/s    in 0.3s    

2025-10-11 21:13:50 (91.1 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [7]:
df_books.head()


,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [8]:
df_ratings.head()


,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [9]:
df = pd.merge(df_ratings,df_books,on='isbn')
df.head()

,user,isbn,rating,title,author
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
1,276726,0155061224,5.0,Rites of Passage,Judith Rae
2,276727,0446520802,0.0,The Notebook,Nicholas Sparks
3,276729,052165615X,3.0,Help!: Level 1,Philip Prowse
4,276729,0521795028,6.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather


In [10]:
combine_book_rating = df.dropna(axis = 0, subset = ['title'])
book_ratingCount = (combine_book_rating.
     groupby(by = ['title'])['rating'].
      count().
      reset_index().
      rename(columns = {'rating': 'totalRatingCount'})
      [['title','totalRatingCount']]
      )
book_ratingCount.head()

,title,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [11]:
rating_with_total_rating = combine_book_rating.merge(book_ratingCount, left_on='title', right_on='title', how='left')
rating_with_total_rating.head()

,user,isbn,rating,title,author,totalRatingCount
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose,60
1,276726,0155061224,5.0,Rites of Passage,Judith Rae,14
2,276727,0446520802,0.0,The Notebook,Nicholas Sparks,650
3,276729,052165615X,3.0,Help!: Level 1,Philip Prowse,1
4,276729,0521795028,6.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,1


In [12]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_ratingCount['totalRatingCount'].describe())

count   241090.000
mean         4.277
std         16.738
min          1.000
25%          1.000
50%          1.000
75%          3.000
max       2502.000
Name: totalRatingCount, dtype: float64


In [13]:
nt = combine_book_rating.merge(book_ratingCount, left_on='title', right_on='title', how='left')
nt.head()

,user,isbn,rating,title,author,totalRatingCount
0,276725,034545104X,0.000,Flesh Tones: A Novel,M. J. Rose,60
1,276726,0155061224,5.000,Rites of Passage,Judith Rae,14
2,276727,0446520802,0.000,The Notebook,Nicholas Sparks,650
3,276729,052165615X,3.000,Help!: Level 1,Philip Prowse,1
4,276729,0521795028,6.000,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,1


In [14]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on='title', right_on='title', how='left')
rating_with_totalRatingCount.head()

,user,isbn,rating,title,author,totalRatingCount
0,276725,034545104X,0.000,Flesh Tones: A Novel,M. J. Rose,60
1,276726,0155061224,5.000,Rites of Passage,Judith Rae,14
2,276727,0446520802,0.000,The Notebook,Nicholas Sparks,650
3,276729,052165615X,3.000,Help!: Level 1,Philip Prowse,1
4,276729,0521795028,6.000,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,1


In [15]:
popularity_treshold = 50
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_treshold')
rating_popular_book.head()

,user,isbn,rating,title,author,totalRatingCount
0,276725,034545104X,0.000,Flesh Tones: A Novel,M. J. Rose,60
2,276727,0446520802,0.000,The Notebook,Nicholas Sparks,650
6,276744,038550120X,7.000,A Painted House,JOHN GRISHAM,838
7,276746,0425115801,0.000,Lightning,Dean R. Koontz,274
8,276746,0449006522,0.000,Manhattan Hunt Club,JOHN SAUL,111


In [16]:
rating_popular_book.shape


(288740, 6)

In [17]:
books_features_df= rating_popular_book.pivot_table(index='title',columns='user',values='rating').fillna(0)
books_features_df.head()

user,8,9,14,16,17,19,23,26,32,39,...,278820,278824,278828,278832,278836,278843,278844,278846,278851,278854
title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
16 Lighthouse Road,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1984,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1st to Die: A Novel,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2010: Odyssey Two,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [18]:
books_features_matrix = csr_matrix(books_features_df.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(books_features_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [22]:
books_features_df.shape

(2444, 47994)

In [23]:
query_index = np.random.choice(books_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(books_features_df.iloc[query_index,:].values.reshape(1,-1), n_neighbors=6)

731


In [24]:
books_features_df.head()

user,8,9,14,16,17,19,23,26,32,39,...,278820,278824,278828,278832,278836,278843,278844,278846,278851,278854
title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
16 Lighthouse Road,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1984,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1st to Die: A Novel,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2010: Odyssey Two,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [ ]:
for i in range(0, len(distances.flatten())):
  if i == 0:
    print('Recommendations for {0}:\n'.format(books_features_df.index[query_index]))
  else:
    print('{0}: {1}, with distance {2}:'.format(i, books_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

ValueError: Please provide a valid book title and DataFrame.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()